In [2]:
import sys
sys.path.append('../')

import pandas as pd
import numpy as np
import re

from mmgclip import preprocess_reports_csv

from hydra import compose, initialize
from omegaconf import DictConfig, OmegaConf
from attrdict import AttrDict

# for auto reload when changes are made in the package
%load_ext autoreload
%autoreload 2

/local/abdel/mmg-clip/env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to /home/abdel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
with initialize(config_path="../configs"):
    cfg = compose(config_name="train_study_reports_clf") # train_sentence_clf, train_multi_class_clf

# convert the config dict to a class object
try:
    cfg['dataset']['config']['lists_dataset_path'] = "../data/02_data_lists/data/lists"
    cfg['dataset']['config']['eliminated_reports_path'] = "../data/reports_lists/eliminate.csv"
    cfg['dataset']['config']['annotated_dataset_path'] = "../data/02_data_T_regions"
    cfg['base']['features_export_dir'] = "../outputs/dataset"
    cfg['base']['export_dir'] = '../outputs/2024-03-11/10-57-24'
    cfg['base']['results_export_dir'] = '../outputs/2024-03-11/10-57-24/results'
    cfg['base']['tensorboard_export_dir'] = '../outputs/...'
    
    mmgconfig = AttrDict(cfg)
    print(mmgconfig)
except Exception as e:
    print("An error occurred:", e)

AttrDict({'checkpoints': {'checkpoints_export_dir': '${hydra:run.dir}/checkpoints', 'checkpoints_file_name': 'model.pth'}, 'base': {'patience': 5, 'seed': 42, 'features_export_dir': '../outputs/dataset', 'export_dir': '../outputs/2024-03-11/10-57-24', 'results_export_dir': '../outputs/2024-03-11/10-57-24/results', 'tensorboard_export_dir': '../outputs/...'}, 'optimizer': {'name': 'adamw', 'config': {'learning_rate': 5e-05, 'weight_decay': 0.0001}}, 'scheduler': {'name': 'cosine', 'config': {'warmup_epochs': 0.1, 'epochs': 30}}, 'experiments': {'config': {'experiment_name': 'classification'}}, 'dataset': {'name': 'StudyReportDataset', 'config': {'enums_class': '', 'search_col': '', 'generate_label_prompt_sentence': False, 'generate_label_prompt_report': True, 'annotated_dataset_path': '../data/02_data_T_regions', 'lists_dataset_path': '../data/02_data_lists/data/lists', 'base_dataset_path': '/storage/Features/features/png_archive/2D_100micron/0/', 'raw_path': 'data/reports/Radboud02_Par

In [4]:
df = pd.read_csv(
    filepath_or_buffer='/storage/Scratch/santiago/project_abdel/clip/data/Radboud02_ParadigmArchiveCorrespondance.csv', 
    encoding="latin1", 
    sep='|', 
    on_bad_lines='skip', 
    names=['raw_id', 'patient_id', 'report_date', 'pathology', 'impression', 'is_malig', 'modality', 'exam_date', 'study_id', 'report', '10'], 
    dtype = str).dropna(axis=1, how='all')

# impression == final assessment

df

,raw_id,patient_id,report_date,pathology,impression,is_malig,modality,exam_date,study_id,report
0,T05-000001,02007775,03/01/2005,Melkgang biopsie mamma links bij tepelvloed: i...,mamma*links*excisie*hyperplasie*geen maligniteit,NotMalignant,MG,2005-01-03,st01,Report current study 2005-01-03: ## 1.3.51.0.1...
1,T05-000031,02002531,03/01/2005,I histologisch naaldbiopt mamma rechts lateraa...,mamma*rechts*naaldbiopt*adenocarcinoom,malignant,MG,2005-01-03,st02,Report current study 2005-01-03: ## 1.3.51.0.1...
2,T05-000108,02007758,04/01/2005,"Mammabiopten links, I solitaire laesie: fibrot...",mamma*links*naaldbiopt*fibrose*microcalcificat...,NotMalignant,MG,2004-12-17,st01,Report current study 2004-12-17: ## 1.3.51.0.1...
3,T05-000109,02002536,04/01/2005,Histologisch naaldbiopt rechter mamma retromam...,mamma*rechts*naaldbiopt*gynaecomastie,NotMalignant,MG,2005-01-04,st02,Report current study 2005-01-04: ## 1.3.51.0.1...
4,T05-000110,02002535,04/01/2005,Histologisch naaldbiopt linker mamma mediaal: ...,mamma*links*naaldbiopt*phyllodes tumor*fibroad...,NotMalignant,MG,2004-12-20,st04,Report current study 2004-12-20: ## 1.3.51.0.1...
...,...,...,...,...,...,...,...,...,...,...
6301,T18-024844,02020654,29/10/2018,<st0>Mamma naaldbiopsie (true cut) links 7-8 u...,mamma*systeem*links*naaldbiopt*geen afwijking*...,NotMalignant,MG,2018-10-29,st00,Report current study 2018-10-29: ## 1.2.840.11...
6302,T18-024958,02019751,30/10/2018,<st0>Lymfklierbiopt axillair rechts met microm...,lymfklier*rechts*biopt*metastase carcinoom*mam...,malignant,MG,2018-10-17,st01,Report current study 2018-10-17: ## 1.2.840.11...
6303,T18-025060,NaN,31/10/2018,<st0>Mamma naaldbiopsie (true cut) links later...,mamma*systeem*links*naaldbiopt*infiltrerend lo...,malignant,MG,2018-10-31,1.2.840.114350.2.357.2.798268.2.133541744.1,Report current study 2018-10-31: ## 1.2.840.11...
6304,T18-025075,NaN,31/10/2018,<st0>Mamma naaldbiopsie (true cut) links media...,mamma*systeem*links*naaldbiopt*geen afwijking,NotMalignant,MG,2018-10-31,1.2.840.114350.2.357.2.798268.2.62926629.1,Report current study 2018-10-31: ## 1.2.840.11...


In [5]:
df['modality'].value_counts()

modality
MG      6261
MGOT      44
CRMG       1
Name: count, dtype: int64

In [6]:
idx = 12

print(df.iloc[idx]['pathology'])
print(df.iloc[idx]['impression'])
# print(df.iloc[:1]['is_malig'][0])

# print(df.iloc[:1]['modality'][0])
# print(df.iloc[:1]['date_2'][0])
# print(df.iloc[:1]['study_id'][0])
print(df.iloc[idx]['report'])

Mammareductie links: hyalinose, ductectasien en plaatselijk apocriene metaplasie. Geen aanwijzingen voor (pre-) maligniteit. Zie ook T05-442 (rechts).
mamma*links*reductie*geen maligniteit
Report current study 2005-01-10: ## 1.3.51.0.1.1.10.11.0.174.20019149.19893    2005-01-10 13:55:17.0000000+02:00 READ Finalized HITGE-BOETESC Medische   gegevens: pre operatieve mammareductie Verslag: er is beiderzijds   geringe scleroserende adenosis. Mammografisch geen aanwijzingen voor   maligniteit. ##  ##


In [7]:
df['is_malig'].value_counts()

is_malig
malignant       3528
NotMalignant    2778
Name: count, dtype: int64

In [8]:
df_processed = preprocess_reports_csv(df, config=mmgconfig, export=False) # export=True
df_processed.head()

,raw_id,patient_id,report_date,pathology,impression,is_malig,exam_date,study_id,report,report_preprocessed,malignancy_benign_section_nl,birads_section_nl,conclusion_nl,labels,has_report_current,has_report_US,has_report_MG,has_report_MR,has_report_others
0,T12-000922,02000002,11/01/2012,Echogeleide histologische naaldbiopsie linker ...,mamma links naaldbiopt fibroadenoom sclerose g...,0,2012-01-11,st02,Report current study 2012-01-11: ## 1.3.51.0.1...,Klinische gegevens. Via bevolkingsonderzoek ve...,Onzeker benigne lesie (d,BI-RADS classificatie-3,Conclusie,"{'birads': '3', 'malignancy': 0, 'masses': {'s...",True,True,False,False,False
1,T07-014170,02000086,08/10/2007,Histologische naaldbiopsie rechter mamma overw...,mamma rechts naaldbiopt fibrose geen maligniteit,0,2007-10-08,st06,Report current study 2007-10-08: ## 1.3.51.0.1...,Jaarlijkse controle in verband met familie ges...,Unknown,BIRADS-classificatie-3,Conclusie Mogelijk minimale toename densiteit...,"{'birads': '3', 'malignancy': 0, 'masses': {'s...",True,True,False,False,False
2,T13-005189,02000092,05/03/2013,Histologisch naaldbiopt aankleurende laesie op...,mamma rechts naaldbiopt cyste geen maligniteit,0,2013-01-21,st08,Report current study 2013-01-21: ## 1.3.51.0.1...,Jaarlijkse controle bij familiaire belasting. ...,Classificatie BI-RADS 2 benigne,Classificatie BI-RADS 2 benigne,Conclusie Status quo,"{'birads': '2', 'malignancy': 0, 'masses': {'s...",True,False,False,False,False
3,T13-006000,02000092,13/03/2013,Naaldbiopten rechter mamma I 12 uur mammaweefs...,mamma rechts biopt ductaal carcinoma in situ i...,1,2013-01-21,st08,Report current study 2013-01-21: ## 1.3.51.0.1...,Jaarlijkse controle bij familiaire belasting. ...,Classificatie BI-RADS 2 benigne,Classificatie BI-RADS 2 benigne,Conclusie Status quo,"{'birads': '2', 'malignancy': 1, 'masses': {'s...",True,False,False,False,False
4,T07-000832,02000098,17/01/2007,Histologische naaldbiopten mediaal onder linke...,mamma links naaldbiopt adenocarcinoom,1,2007-01-17,st03,Report current study 2007-01-17: ## 1.3.51.0.1...,last pleurodynie links. Palpabele cyste latera...,Laesie suspect voor maligniteit,"BIRADS 4, aanvullende biopsie geÃ¯ndiceerd",Unknown,"{'birads': '4', 'malignancy': 1, 'masses': {'s...",True,True,False,False,False


In [9]:
df_processed.shape

(2075, 19)

In [10]:
# birads in roman [1, 9, 391, 751]
# 1621 is too long

# idx = 1907
# idx = 433

idx = 1331

# impression
# report_preprocesed is same as findings?? seems so*
# pass the impression to the model, then correlate the text logits?*

print(df_processed.iloc[idx].is_malig)
print(df_processed.iloc[idx].report)
print(df_processed.iloc[idx].report_preprocessed)
print(df_processed.iloc[idx].impression)
print(df_processed.iloc[idx].malignancy_benign_section_nl)
print(df_processed.iloc[idx].birads_section_nl)
print(df_processed.iloc[idx].conclusion_nl)
print(df_processed.iloc[idx].labels)

1
Report current study 2012-04-23: ## 1.3.51.0.1.1.10.11.0.174.21346525.1347386  2012-04-23   09:11:00.0000000+01:00 Finalized Medische gegevens: Via   bevolkingsonderzoek verdichting rechts lateraal boven. BI-RADS 0. Zelf   niets gevoeld. Bij lichamelijk onderzoek geen palpabele afwijkingen   mammae of axillae beiderzijds. Familieanamnese negatief. X-mammografie   vergeleken met 9 juli 2010 en eerder. Redelijk goed beoordeelbaar   mammogram. Mammaklier percentage ongeveer 25%, ACR 1. Thans nieuwe   verdichting in het laterale bovenkwadrant van de rechter mamma met een   diameter van ongeveer 8 mm, deze verstrijkt bij compressie. Ook bekende   redelijk scherp afgrensbare ronde densiteit in het laterale   bovenkwadrant van de linker mamma waarbij nu toename van   microcalcificaties in de meest ventrale lesie op de CC-opname. Geen   afwijkingen in de cutis of subcutis. Echografie mamma beiderzijds. Geen   onderzoek ter vergelijking. Rechts: In het laterale bovenkwadrant van   de rechter 

How many sections of MG reports are available?

In [11]:
len(df_processed[df_processed['has_report_MG'] == True])

206

In [23]:
df_processed[df_processed['has_report_MG'] == True].iloc[22].report_preprocessed

'Erwtgrote pijnloze vaste zwelling op 7 uur van rechter tepelhof. 2004 mastitis met abces rechterborst. Familieanamnese geen bijzonderheden. Verslag Mammografisch onderzoek beiderzijds in twee richtingen. Geen eerder onderzoek ter vergelijking. Goed beoordeelbaar mammogram, 0-25% klierweefsel. Verdachte beschaduwingen laterale onderkwadrant rechts circa 24 mm. Geen overige verdachte beschaduwingen. Normaal aspect van tepel en huid beiderzijds. Aansluitend echografisch onderzoek toont rechts op circa 7 uur een hypo-echogene onscherp afgrensbare laesie van circa 20 x 16 mm. Circa 15 mm latero craniaal hiervan een tweede laesie van circa 8 x 5 mm. Beide laesies zijn palpabel. De grootste laesie werd aangeprikt 14G biopt. Materiaal opgestuurd voor PA diagnostiek. Conclusie BIRADS V. Verdachte laesie rechts lateraal onder. Materiaal opgestuurd voor PA diagnostiek. Afspraak voor uitslag werd gemaakt met de mammacare.  Grootste laesie in het geheel in het preparaat aanwezig. Tweede kleinere l

In [63]:
df_processed.sample().iloc[0]

raw_id                                                                 T11-021902
patient_id                                                               02001787
report_date                                                            14/10/2011
pathology                       Echogeleide histologische naaldbiopsie lateral...
impression                                  mamma links naaldbiopt adenocarcinoom
is_malig                                                                        1
exam_date                                                              2011-10-14
study_id                                                                     st02
report                          Report current study 2011-10-14: ## 1.3.51.0.1...
report_preprocessed             Sinds een week zwelling linker mamma. Familiea...
malignancy_benign_section_nl    Het beeld is toch sterk verdacht voor een mali...
birads_section_nl                                Classificatie BI-RADS 5 maligne.
conclusion_nl   

In [65]:
# df_processed[(df_processed['raw_id'] == "T13-012478") & (df_processed['patient_id'] == "02000828") & (df_processed['report_date'] == "05/06/2013")]

In [17]:
df_processed['labels'].apply(lambda x: x.get('birads')).value_counts()

labels
5          713
4          581
3          261
unknown    218
6           89
2           88
0           77
1           60
Name: count, dtype: int64

In [13]:
filtered_rows = df_processed[df_processed['labels'].apply(lambda x: x.get('calcifications')['distribution'] != "Unknown")]
# print(np.unique(filtered_rows.labels))
# filtered_rows.iloc[6]
filtered_label = [l['calcifications']['distribution'] for l in filtered_rows.labels.values]
np.unique(filtered_label)

KeyError: 'birads'

Are there any duplicate patients?

In [44]:
# out of the 2183 rows, only 1268 are unique, thus many duplicate occurances
print(len(np.unique(df_processed['patient_id'])))

1268


Are there any duplicate reports?

In [45]:
# out of the 2183 rows, only 1385 are unique, thus many duplicate occurances
print(len(np.unique(df_processed['report'])))

1507


From the duplicate patients ids, do they have the same studies?

In [47]:
df_processed[df_processed['patient_id'] == '02000092']

,raw_id,patient_id,report_date,pathology,impression,is_malig,exam_date,study_id,report,report_preprocessed
2,T13-005189,02000092,05/03/2013,Histologisch naaldbiopt aankleurende laesie op...,mamma rechts naaldbiopt cyste geen maligniteit,0,2013-01-21,st08,Report current study 2013-01-21: ## 1.3.51.0.1...,Jaarlijkse controle bij familiaire belasting. ...
3,T13-006000,02000092,13/03/2013,Naaldbiopten rechter mamma I 12 uur mammaweefs...,mamma rechts biopt ductaal carcinoma in situ i...,1,2013-01-21,st08,Report current study 2013-01-21: ## 1.3.51.0.1...,Jaarlijkse controle bij familiaire belasting. ...


In [319]:
unique_dataset = []

for idx, patient_id in enumerate(np.unique(df_processed['patient_id'].values)):
    patient_reports = df_processed[df_processed['patient_id'] == patient_id]

    # if there are more than one report per patient
    if len(patient_reports) > 1:
        len_unique_findings = len(np.unique(patient_reports['findings']))
        n_reports = len(patient_reports)
        len_unique_impression = len(np.unique(patient_reports['impression']))
        len_unique_is_malig = len(np.unique(patient_reports['is_malig']))
        len_unique_study_id = len(np.unique(patient_reports['study_id']))
        len_unique_report = len(np.unique(patient_reports['study_id']))
        
        unique_dataset.append([patient_id, n_reports, len_unique_findings, len_unique_impression, len_unique_is_malig, len_unique_study_id, len_unique_report])
    else:
        unique_dataset.append([patient_id, 1, 1, 1, 1, 1, 1])

df_unique_dataset = pd.DataFrame(unique_dataset, columns=['patient_id', 'n_reports', 'len_unique_findings', 'len_unique_impression', 'len_unique_is_malig', 'len_unique_study_id', 'len_unique_report'])

In [320]:
df_unique_dataset

,patient_id,n_reports,len_unique_findings,len_unique_impression,len_unique_is_malig,len_unique_study_id,len_unique_report
0,02000002,1,1,1,1,1,1
1,02000086,1,1,1,1,1,1
2,02000092,2,2,2,2,1,1
3,02000098,2,2,2,1,1,1
4,02000113,1,1,1,1,1,1
...,...,...,...,...,...,...,...
1263,02020541,1,1,1,1,1,1
1264,02020546,2,2,2,2,1,1
1265,02020570,1,1,1,1,1,1
1266,02020611,1,1,1,1,1,1


Check if there patients with more than one report

In [321]:
print(np.unique(df_unique_dataset['len_unique_study_id'].values))
print(np.unique(df_unique_dataset['len_unique_report'].values))

[1 2 3 4]
[1 2 3 4]


In [322]:
df_unique_dataset[df_unique_dataset['len_unique_report']  == 4]

,patient_id,n_reports,len_unique_findings,len_unique_impression,len_unique_is_malig,len_unique_study_id,len_unique_report
17,02000370,5,5,4,2,4,4
412,02002806,5,5,5,2,4,4
658,02003369,4,4,4,1,4,4


In [323]:
# Four different reports, four different studies 
case = df_processed[df_processed['patient_id'] == '02002806']
case

,raw_id,patient_id,report_date,findings,impression,is_malig,study_date,study_id,report
811,T05-013237,02002806,25/10/2005,Mammabiopsie links slechts zeer beperkt materi...,mamma links biopt onvoldoende materiaal,1,2005-10-25,st01,Status na mammasparende therapie links. Mammog...
812,T08-000761,02002806,16/01/2008,Type excisie Ablatio mamma links. Indien beken...,mamma links ablatio invasief ductaal carcinoom...,1,2007-12-18,st02,Voorgeschiedenis mammasparende therapie links ...
813,T14-001039,02002806,14/01/2014,Echogeleide histologische naaldbiopsie linker ...,mamma links naaldbiopt cyste verhoorning geen ...,0,2014-01-14,st03,Relevante voorgeschiedenis ablatio 2008-2000-1...
814,T14-001040,02002806,14/01/2014,Echogeleide histologische naaldbiopsie rechter...,mamma rechts naaldbiopt intraductaal papillair...,1,2014-01-14,st03,Relevante voorgeschiedenis ablatio 2008-2000-1...
815,T17-006282,02002806,13/03/2017,Stansbiopt linker mamma huidbiopt met in de de...,huid mamma links biopt carcinoom ip reactie in...,0,2017-02-20,st08,Controle. Status na ablatio links en mammaspar...


In [324]:
case.iloc[3].findings

'Echogeleide histologische naaldbiopsie rechter mamma het beeld past het best bij een intraductaal papillair carcinoom. Invasieve groei is op dit materiaal niet uit te sluiten. '

There are patients of reports, that has unique or different report